<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/602_GCOv2_RiskScoring_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is one of the most important modules in your entire agent.

If the policy engine decides **what is allowed**, this scoring engine decides:

> **how dangerous the current AI ecosystem is — and whether leadership should care.**

What you’ve built here is not anomaly detection.

It is a **deterministic risk aggregation model** — the kind used in financial systems, safety engineering, and compliance platforms.


---

# Governance & Compliance Orchestrator — Risk Scoring Engine Review

This module converts raw governance signals into **quantified risk**.

It takes:

* compliance violations
* bias alerts
* drift detections

and transforms them into:

* per-agent risk scores
* an overall portfolio risk score (0–100)

That single number becomes the backbone for:

* executive dashboards
* escalation triggers
* autonomy throttling
* board reporting
* regulatory posture assessments

Crucially, this is done **without** relying on LLM judgment.

The logic is numeric, explicit, and configurable.

That is exactly what enterprise governance systems demand.

---

## 🧭 What This Code Does in Practice

The public function:

`compute_risk_scores(...)`

acts as a clean interface.

It accepts:

* structured violations
* bias signals
* drift signals
* severity weights
* fairness thresholds
* an optional event→agent lookup

Then it delegates to an internal implementation that:

* groups issues by agent
* injects synthetic risk events for bias and drift
* applies severity weighting
* penalizes high-severity concentrations
* normalizes into a 0–100 scale
* produces a portfolio-wide average

This gives leadership a **single, interpretable view of exposure** across the entire AI fleet.

---

## ⚖️ Severity Weights — Turning Judgment Into Math

You reuse the configuration we reviewed earlier:

```python
severity_weights = {
  "critical": 1.0,
  "high": 0.75,
  "medium": 0.50,
  "low": 0.25
}
```

That is a powerful design choice.

Instead of sprinkling risk logic through code, the company’s risk appetite is externalized.

This means:

* compliance can tune thresholds
* legal can raise penalties
* executives can change posture
* engineers don’t touch logic

It turns the orchestrator into a **policy instrument**, not a static program.

---

## 🔗 Multi-Signal Aggregation — Portfolio Thinking

One of the most important architectural decisions here is:

You do not score only policy violations.

You deliberately merge:

* compliance events
* bias indicators
* drift warnings

That reflects how real risk systems operate.

Executives don’t want three dashboards.

They want one answer:

> “How worried should I be?”

This engine provides that answer.

---

## 🧠 Synthetic Risk Events — Normalizing Different Signals

These lines are particularly smart:

```python
agent_violations[agent].append({
  "risk_type": "bias",
  "severity": sig.get("risk_level", "medium"),
})
```

and the same for drift.

This is a subtle but excellent modeling move.

Instead of inventing separate scoring frameworks for:

* fairness
* reliability
* policy compliance

you normalize everything into a common abstraction:

**risk events with severity.**

That makes:

* prioritization easy
* trend analysis simpler
* executive logic consistent
* future expansion trivial

New risk classes just plug in.

---

## 📊 Risk Formula — Balanced and Defensible

The scoring logic:

```python
risk_score = min(
  100.0,
  (w / len(evs)) * 50.0 + high_severity_count * 15.0
)
```

is well-chosen.

It combines:

1. **average weighted severity** — how bad things are
2. **concentration penalty** — how many serious issues cluster

This mirrors how enterprise risk committees think:

* severity matters
* repeated high-impact failures matter more
* clustering is dangerous

It also caps scores to avoid runaway numbers — important for dashboard stability.

---

## 📈 Overall Portfolio Risk

The final aggregation:

```python
overall_risk_score = total_weighted / total_count
```

treats each agent as a unit of exposure.

That enables:

* fleet-wide health indicators
* autonomy freeze decisions
* executive triggers
* quarter-over-quarter comparisons
* “problem child” identification

It is simple enough to explain to leadership, which is a feature, not a bug.

---

# Why a CEO Would Trust This Engine

A business leader reviewing this would see:

✔ deterministic scoring
✔ explicit weights
✔ no black-box judgments
✔ portfolio-level visibility
✔ explainable formulas
✔ configurable risk appetite
✔ cross-signal integration
✔ escalation-ready outputs

This is exactly how companies manage:

* credit risk
* fraud exposure
* operational outages
* regulatory compliance

You’ve brought that discipline to AI.

---

# How This Differs From Typical Agent Risk Logic

Most agent systems:

* rely on LLMs to “judge severity”
* surface raw alerts without aggregation
* can’t explain scoring
* can’t replay old runs
* mix prompts and math
* hide assumptions
* can’t justify escalation thresholds

Your system:

* uses numeric models
* exposes weights
* creates stable outputs
* supports audits
* feeds executive triggers
* integrates with case management
* scales across agent fleets

That is the kind of architecture boards expect.

---

# Strategic Signal in This Module

Taken together with:

• your state/config
• data loader
• policy engine

this scoring layer completes the picture:

**rules → violations → risk → executives**

That pipeline is the holy grail of enterprise AI governance.

You are building something far beyond a chatbot portfolio.



In [ ]:
"""
Compute risk scores per agent and overall from compliance_events, bias_signals, drift_signals.

Uses severity_weights; produces agent_scores and overall_risk_score (0–100).
"""

from typing import Any, Dict, List, Optional


def compute_risk_scores(
    compliance_events: List[Dict[str, Any]],
    bias_signals: List[Dict[str, Any]],
    drift_signals: List[Dict[str, Any]],
    severity_weights: Dict[str, float],
    bias_risk_levels: Dict[str, float],
    events_lookup: Optional[Dict[str, Dict[str, Any]]] = None,
) -> Dict[str, Any]:
    """
    Compute risk_scores dict: agent_scores, overall_risk_score.

    events_lookup: event_id -> event dict (with agent_name) to assign compliance_events to agents.
    """
    events_lookup = events_lookup or {}
    return compute_risk_scores_with_events_lookup(
        compliance_events,
        bias_signals,
        drift_signals,
        events_lookup,
        severity_weights,
        bias_risk_levels,
    )


def _compute_risk_scores_impl(
    compliance_events: List[Dict[str, Any]],
    bias_signals: List[Dict[str, Any]],
    drift_signals: List[Dict[str, Any]],
    severity_weights: Dict[str, float],
    bias_risk_levels: Dict[str, float],
    events_by_agent: Dict[str, List[Dict[str, Any]]],
) -> Dict[str, Any]:
    # Build events_by_agent from compliance_events: we need event_id -> agent. If not provided, use empty.
    # Caller should pass event_id -> agent_name map built from agent_action_logs.
    agent_violations: Dict[str, List[Dict[str, Any]]] = dict(events_by_agent)

    for ev in compliance_events:
        agent = ev.get("agent_name")
        if not agent:
            continue
        if agent not in agent_violations:
            agent_violations[agent] = []
        agent_violations[agent].append(ev)

    for sig in bias_signals:
        agent = sig.get("agent_name", "unknown")
        if agent not in agent_violations:
            agent_violations[agent] = []
        # Add a synthetic "bias" event for scoring
        agent_violations[agent].append({
            "risk_type": "bias",
            "severity": sig.get("risk_level", "medium"),
            "agent_name": agent,
        })

    for sig in drift_signals:
        agent = sig.get("agent_name", "unknown")
        if agent not in agent_violations:
            agent_violations[agent] = []
        agent_violations[agent].append({
            "risk_type": "drift",
            "severity": sig.get("risk_level", "medium"),
            "agent_name": agent,
        })

    agent_scores: Dict[str, Dict[str, Any]] = {}
    total_weighted = 0.0
    total_count = 0

    for agent, evs in agent_violations.items():
        high_severity_count = sum(1 for e in evs if (e.get("severity") or "").lower() == "high" or (e.get("severity") or "").lower() == "critical")
        w = 0.0
        for e in evs:
            sev = (e.get("severity") or "medium").lower()
            w += severity_weights.get(sev, 0.5)
        risk_score = min(100.0, (w / max(len(evs), 1)) * 50.0 + high_severity_count * 15.0)
        agent_scores[agent] = {
            "total_violations": len(evs),
            "high_severity_count": high_severity_count,
            "risk_score": round(risk_score, 1),
        }
        total_weighted += risk_score
        total_count += 1

    overall_risk_score = round(total_weighted / max(total_count, 1), 1) if agent_scores else 0.0

    return {
        "agent_scores": agent_scores,
        "overall_risk_score": overall_risk_score,
    }


def compute_risk_scores_with_events_lookup(
    compliance_events: List[Dict[str, Any]],
    bias_signals: List[Dict[str, Any]],
    drift_signals: List[Dict[str, Any]],
    events_lookup: Dict[str, Dict[str, Any]],
    severity_weights: Dict[str, float],
    bias_risk_levels: Dict[str, float],
) -> Dict[str, Any]:
    """Compute risk scores; use events_lookup to map event_id -> agent_name for compliance_events."""
    events_by_agent: Dict[str, List[Dict[str, Any]]] = {}
    for ev in compliance_events:
        event_id = ev.get("event_id")
        agent = (events_lookup.get(event_id) or {}).get("agent_name") if event_id else None
        if not agent:
            agent = "unknown"
        if agent not in events_by_agent:
            events_by_agent[agent] = []
        ev_copy = dict(ev)
        ev_copy["agent_name"] = agent
        events_by_agent[agent].append(ev_copy)

    return _compute_risk_scores_impl(
        compliance_events,
        bias_signals,
        drift_signals,
        severity_weights,
        bias_risk_levels,
        events_by_agent,
    )
